In [1]:
import pandas as pd

In [ ]:
df_test = pd.read_csv(r"C:\Users\leila\Downloads\projet pfa\house-price-prediction\data\raw data\test.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw data/test.csv'

In [3]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [5]:
# 1. Appliquer les mêmes traitements aux colonnes catégorielles
na_cols = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
           'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 
           'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']
df_test[na_cols] = df_test[na_cols].fillna('NA')

df_test['MasVnrType'] = df_test['MasVnrType'].fillna('None')

# 2. Imputation numérique cohérente avec df_train
# Utiliser les médianes calculées sur df_train pour LotFrontage
neighborhood_medians = df_test.groupby('Neighborhood')['LotFrontage'].median()
df_test['LotFrontage'] = df_test.apply(
    lambda row: neighborhood_medians[row['Neighborhood']] if pd.isna(row['LotFrontage']) else row['LotFrontage'], 
    axis=1
)

# Remplacer les autres NA numériques
numeric_zero_cols = ['MasVnrArea', 'GarageYrBlt', 'BsmtFinSF1', 'BsmtFinSF2', 
                    'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                    'GarageCars', 'GarageArea']
df_test[numeric_zero_cols] = df_test[numeric_zero_cols].fillna(0)

# 3. Imputer les modes calculés sur df_train pour les autres catégories
cat_cols_to_impute = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd',
                     'Electrical', 'KitchenQual', 'Functional', 'SaleType']

for col in cat_cols_to_impute:
    mode_val = df_test[col].mode()[0]
    df_test[col] = df_test[col].fillna(mode_val)

# 4. Vérification des relations entre variables
# Si pas de sous-sol
bsmt_cols = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 
             'BsmtFullBath', 'BsmtHalfBath']
df_test.loc[df_test['BsmtQual'] == 'NA', bsmt_cols] = 0

# Si pas de garage
df_test.loc[df_test['GarageType'] == 'NA', ['GarageCars', 'GarageArea']] = 0

# 5. Vérification finale
print("Valeurs manquantes restantes dans df_test :")
print(df_test.isnull().sum().sum())  

Valeurs manquantes restantes dans df_test :
0


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1459 non-null   object 
 3   LotFrontage    1459 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          1459 non-null   object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1459 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [7]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NA,Reg,Lvl,AllPub,...,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,MnPrv,NA,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,NA,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NA,IR1,HLS,AllPub,...,144,0,NA,NA,NA,0,1,2010,WD,Normal


In [8]:
from sklearn.preprocessing import LabelEncoder

# ---------------------------------------------------------------
# 2. Ordinal Encoding (manuelle)
# ---------------------------------------------------------------

ordinal_mapping = {
    'LotShape': {'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1},
    'Utilities': {'AllPub': 4, 'NoSewr': 3, 'NoSeWa': 2, 'ELO': 1},
    'LandSlope': {'Gtl': 1, 'Mod': 2, 'Sev': 3},
    'ExterQual': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1},
    'ExterCond': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1},
    'BsmtQual': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0},
    'BsmtCond': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0},
    'BsmtExposure': {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0},
    'BsmtFinType1': {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0},
    'BsmtFinType2': {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0},
    'HeatingQC': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
}

for col, mapping in ordinal_mapping.items():
    df_test[col] = df_test[col].map(mapping)

# ---------------------------------------------------------------
# 2. One-Hot Encoding (variables nominales)
# ---------------------------------------------------------------
onehot_cols = [
    'MSZoning', 'Alley', 'LandContour', 'LotConfig',
    'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'RoofStyle',
    'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
    'Heating', 'MSSubClass'  # MSSubClass est nominale !
]

df_test = pd.get_dummies(df_test, columns=onehot_cols, drop_first=True)

# ---------------------------------------------------------------
# 3. Encodage binaire pour les variables à 2 catégories
# ---------------------------------------------------------------
# Variable 'Street' 
df_test['Street'] = df_test['Street'].map({'Pave': 1, 'Grvl': 0})

In [9]:
print(df_test.head())


     Id  LotFrontage  LotArea  Street  LotShape  Utilities  LandSlope  \
0  1461         80.0    11622       1         4          4          1   
1  1462         81.0    14267       1         3          4          1   
2  1463         74.0    13830       1         3          4          1   
3  1464         78.0     9978       1         3          4          1   
4  1465         43.0     5005       1         3          4          1   

  HouseStyle  OverallQual  OverallCond  ...  MSSubClass_70  MSSubClass_75  \
0     1Story            5            6  ...          False          False   
1     1Story            6            6  ...          False          False   
2     2Story            5            5  ...          False          False   
3     2Story            6            6  ...          False          False   
4     1Story            8            5  ...          False          False   

   MSSubClass_80  MSSubClass_85  MSSubClass_90  MSSubClass_120  \
0          False          False 